# 판례 임베딩
- 사용자의 입력과 코사인 유사도를 비교하기 위한 판례 임베딩
- 판례를 미리 임베딩하면 유사도를 비교하는데 드는 시간을 절약할 수 있다.

## 1. 패키지 로드

In [1]:
import pandas as pd
import numpy as np

from gensim import models
from ko_preprocessing import K_preprocessing
from prec_preprocessing import PrecedentPrcs

## 2. 판례 데이터 로드

In [2]:
prec_df = pd.read_csv("../data/prec_data.csv", encoding='utf-8-sig')
prec_df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,20220415.0,선고,수원고법,400202.0,형사,400102.0,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,NaN,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,20220411.0,자,서울고등법원,400202.0,가사,400103.0,결정,NaN,NaN,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",NaN,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,20220323.0,선고,대구고법,400202.0,민사,400101.0,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",NaN,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,20220322.0,자,대법원,400201.0,가사,400103.0,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),NaN,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",NaN,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,20220317.0,선고,대법원,400201.0,민사,400101.0,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n","대법원 2018. 11. 29. 선고 2016다266606, 266613 판결(공2...","【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


## 3. FastText 모델 로드

In [3]:
fasttext_model = models.fasttext.load_facebook_model('../FastText/pre_trained_fasttext/cc.ko.300.bin')

[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xa1\x9c'; replacing invalid characters, using '\\xed\\xa1\\x9c'
[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xb0\x80'; replacing invalid characters, using '\\xed\\xb0\\x80'
[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xa7\x80'; replacing invalid characters, using '\\xed\\xa7\\x80'
[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xb8\xb0'; replacing invalid characters, using '\\xed\\xb8\\xb0'
[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xa0\x9c'; replacing invalid characters, using '\\xed\\xa0\\x9c'
[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xa6\xac'; replacing invalid characters, using '\\xed\\xa6\\xac'
[Korean Sentence Splitter]: failed to decode invalid unicode bytes b'\xed\xb3\xb5'; replacing invalid characters, using '\\xed\\xb3\\xb5'
[Korean Sentence Splitter]: failed

## 4. 임베딩

### 1) 판례 기본 전처리

In [4]:
prec_preprocess_obj = PrecedentPrcs()

In [5]:
pr_prec_df = prec_preprocess_obj.run(prec_df)

D:\Judg-Precedent-project\version_0.1\Embedding\prec_preprocessing.py:69: FutureWarning: The default value of regex will change from True to False in a future version.
  df['선고일자'] = df['선고일자'].str.replace('[^0-9]', '')
D:\Judg-Precedent-project\version_0.1\Embedding\prec_preprocessing.py:92: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 92 of the file D:\Judg-Precedent-project\version_0.1\Embedding\prec_preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(x)


In [6]:
pr_prec_df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,None,선고,수원고법,400202,형사,400102,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,없음,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,None,자,서울고등법원,400202,가사,400103,결정,미기재,미기재,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",없음,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,None,선고,대구고법,400202,민사,400101,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",없음,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,None,자,대법원,400201,가사,400103,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),미기재,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",없음,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,None,선고,대법원,400201,민사,400101,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n",,"【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


### 2) 문장 토큰화

In [7]:
%time
prec_tokened_sentences = pr_prec_df['판례내용'].apply(K_preprocessing.tokenize_sent)

[Korean Sentence Splitter]: Initializing Pynori...


Wall time: 0 ns


KeyboardInterrupt: 

### 3) 한글, 숫자외의 문자 제거

In [10]:
%%time
K_preprocessing.tokenize_sent(pr_prec_df['판례내용'][0])

Wall time: 339 ms


['【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 변호사 손영서【원심판결】 수원지법 안양지원 2021. 10. 8. 선고 2021고합118 판결【주 문】 원심판결(이유무죄 부분 포함)을 파기한다.',
 '피고인을 징역 2년에 처한다.',
 '다만 이 판결 확정일로부터 3년간 위 형의 집행을 유예한다.',
 '피고인에게 40시간의 성폭력 치료강의 수강을 명한다.',
 '피고인에 대하여 아동·청소년 관련기관 등과 장애인복지시설에 5년간 취업제한(운영 및 사실상 노무제공 금지 포함)을 명한다.【이 유】 1. 항소이유의 요지 가.',
 '피고인(양형부당) 원심이 피고인에게 선고한 형(징역 2년 6월 등)은 너무 무거워서 부당하다.',
 '나. 검사 1) 사실오인(형법 제305조 제1항의 미성년자의제강간의 점) 피해자가 당시 착용하고 있던 의상은 전형적으로 초등학생의 의상이었고, 피해자의 외모 역시 평범한 초등학생으로 보이며, 피해자는 피고인을 만나기 전 피고인에게 "나 아빠랑 가도 돼?"라고 물어보았는바, 상식적으로 피해자가 매우 어리다고 추정할 수 있는 등 피고인이 당시 피해자가 13세 미만이라는 사실을 인식하고 있었다고 봄이 상당하다.',
 '그럼에도 이와 달리 피고인이 이 사건 당시 피해자가 13세 미만인 사실을 미필적으로라도 인식하고 있었음이 증명되지 않는다는 이유로 형법 제305조 제1항의 미성년자의제강간의 점에 대하여 이유무죄를 선고한 원심판결에는 사실오인의 위법이 있다. 2) 법리오해(형법 제305조 제2항의 미성년자의제강간의 점) 설령 피고인이 피해자의 나이를 13세 미만이라고 인식하지 못하였다고 하더라도, 피고인은 실제 13세 미만인 피해자를 13세 이상 16세 미만 정도로 인식하고 간음하였는바, 이는 경한 고의로 중한 결과가 발생한 경우에 해당하고 형법 제305조 제2항의 신설 취지, 일반인들의 법감정 등을 고려하면 형법 제305조 제2항의 미성년자의제강간죄의 기수범으로 처벌하여야 함에도 불능미수 유죄로만 인정한 원심판